The NeRF paper [@Mildenhall2020], gives 



### Dataset
Get the tiny blender nerf dataset

In [ ]:
#| code-fold: true
import os 
if not os.path.exists('/tmp/tiny_nerf_data.npz'):
    !wget http://cseweb.ucsd.edu/~viscomp/projects/LF/papers/ECCV20/nerf/tiny_nerf_data.npz -P /tmp

What does the data look like?

In [ ]:
import numpy as np
data = np.load('/tmp/tiny_nerf_data.npz', allow_pickle=True)
images = data['images']
poses = data['poses']
focal = data['focal']

print(f'Images size : {images.shape}')
print(f'Pose : {poses[0]}')
print(f'Focal length: {focal}')

The pose includes the extrinsic parameters - rotation and translation matrices - $[R | T]$. And the focal length is the intrinsic property. Using this we can find the camera ray:


$$
\vec{r} = \vec{o} + t\vec{d}
$$ {#eq-camera-ray}


The vector $\vec{d}$, can be thought of as $\vec{d}= R\vec{d}_o$, where $\vec{d}_o$ is the direction vector of a standard pinhole camera without rotation - which can be found by using the focal length: $\vec{d}_o = f[x/f, y/f, -1]$


The cones in the below plot shows the direction of camera ray. The original is shown where $z=0$, and two additional poses are also plotted.

In [ ]:
#| code-fold: true
import plotly.graph_objects as go
#xv, yv = np.meshgrid(0, 500, indexing='xy')
H = 100
W = 100
x, y, z = np.mgrid[0:W:10, 0:H:10, 0:H:10]
x = x.flatten() - W/2
y = y.flatten() - H/2
z = z.flatten()*0

u = x/focal 
v = y/focal

w = -np.ones_like(x)
print(u[0], v[0], w[0])

fig = go.Figure()
fig.add_trace(go.Cone(
    x=x,
    y=y,
    z=z,
    u=u,
    v=v,
    w=w,
    ))

for i in range(0, 40,30):
    rot = poses[i][:3,:3]
    t = poses[i][:3,3]
    t= t[..., np.newaxis]
    p = np.stack([x, y, z]) 
    d = np.stack([u, v, w]) 
    p = rot @ p 
    d = rot @ d 
    nx, ny, nz = p 
    nu, nv, nw = d 
    fig.add_trace(go.Cone(
        x=nx,
        y=ny,
        z=nz,
        u=nu,
        v=nv,
        w=nw*2,
        ))

fig.show()

In [ ]:
import jax
import jax.numpy as jnp

def get_ray(H, W, focal, pose):
    x, y = jnp.mgrid[0:W, 0:H]
    x = x - W/2
    y = y - H/2
    y = -y # bender seems to use -y 

    x = x.flatten()
    y = y.flatten()

    direction = jnp.stack([x, y, -jnp.ones_like(x)])
    # Normalize direction
    direction_norm = jnp.linalg.norm(direction, ord=2, axis=0)
    direction = direction/direction_norm

    rot = pose[:3, :3] 
    direction = jnp.matmul(rot, direction)

    translation = pose[:3, 3]
    translation = translation[..., jnp.newaxis]
    origin = jnp.broadcast_to(translation, direction.shape)
    return origin, direction 

def get_ray_concrete(x):
    return get_ray(H, W, focal, x)

get_ray_batched = jax.vmap(get_ray_concrete, in_axes=(0))

Let's see how much `vmap` speeds up `get_ray`:


In [ ]:
%%timeit
for _ in range(2):
    for pose in poses: _, _ = get_ray_concrete(pose)

In [ ]:
%%timeit
for _ in range(2):
    _, _ = get_ray_batched(poses)

What about using `jit`?

In [ ]:
%%timeit
get_ray_batched_jit = jax.jit(get_ray_batched)
for _ in range(2):
    direction, origin = get_ray_batched_jit(poses)

Using `vmap` is fast, but `jit` is much much faster!

### Camera ray and position encoding
Let's construct the function @eq-camera-ray.


In [ ]:
near = 2.
far = 6.
num_samples = 64

def r_func(o, d, t):
    r = o + t*d
    return r

t = jnp.linspace(near, far, num_samples)

get_ray_batched_jit = jax.jit(get_ray_batched)
origin, direction = get_ray_batched_jit(poses)
r = r_func(origin, direction, num_samples)
print(r.shape)

In **Section 5.1** the position encoding $\gamma$ is a mapping from $\mathbb{R}$ to $\mathbb{R}^{2L}$, where $L=10$ for the coordinate values $x$ and $l=4$ the direction vector :


$$
\gamma(p) = (sin(2^0\pi p), cos(2^0 p), \dots, sin(2^{L-1}\pi p), cos(2^{L-1} p))
$$

In [ ]:
def encoding_func(x, L):
    encoded_array = []
    for i in range(L):
        encoded_array.extend([jnp.sin(jnp.power(2, i) * jnp.pi * x), jnp.cos(jnp.power(2,i) * jnp.pi * x)])
    return jnp.array(encoded_array)

encoding_func_batched = jax.vmap(encoding_func, (0, None))
r_encoded = encoding_func_batched(r, 10)
print(f'Position before encoding shape {r.shape}')
print(f'Position after encoding shape {r_encoded.shape}')

### The render function

Equation (3) in the paper is:


$$
\hat{C} \vec(r) = \sum_{i=1}^{N} T_i (1- \text{exp}(-\sigma_i \delta_i))\vec{c}_i, \text{where}, T_i=\text{exp}\left( \sum_{j=1}^{i-1} \sigma_j \delta_j \right)
$$


```python
def render(model_func, poses, near, far, num_samples):
    origins, directions = get_ray_batched(poses)

    t = jnp.linspace(near, far, num_samples)
    r = r_func(origins, directions, num_samples)

    r = jnp.reshape(r, [-1, 3])
    model_func(r)

```
### Construct the network

The network is a series of MLPs


In [ ]:
from jax import random
def get_params(m, n, key, scale=1e-2):
    w_key, b_key = random.split(key)
    return scale * random.normal(w_key, (n, m)), scale * random.normal(b_key, (n,))

def get_all_params(sizes, key):
    keys = random.split(key, len(sizes))
    param_array = []
    for i in range(len(sizes) - 1):
        m = sizes[i]
        n = sizes[i + 1]
        param_array.append(get_params(m, n, key))
    return param_array  
